# MongoDB

## 1. Mongo setup

### 1.1. Utils

In [ ]:
from bson import json_util as json 
from pymongo.cursor import Cursor
from bson import datetime
import pytz

def format_result(rs):
    return '\t{}'.format(json.dumps(rs, indent=2).replace('\n', '\n\t'))

def date_to_datetime(date):
    return datetime.datetime.combine(date, datetime.datetime.min.time()).replace(tzinfo=pytz.utc)

### 1.2. Make connection

In [ ]:
from pymongo import MongoClient

mongo = MongoClient(host='localhost', port=27017, maxPoolSize=5)
print('* mongodb connect success: {}'.format(mongo))

### 1.3. Get server status

In [ ]:
connection_addr = mongo.address
print('* connection address is: "{}:{}"'.format(*connection_addr))

info = mongo.admin.command('serverStatus')
print('* connections: current: {current}, available: {available}, total: {totalCreated}, activie: {active}'.format(**info['connections']))

## 2. Database

### 2.1. Create database

In [ ]:
db = mongo['test_db']

### 2.2. Delete database

In [ ]:
db.command('dropDatabase')

### 2.3. List database

In [ ]:
db_names = mongo.list_database_names()
print('* contain databases are: {}'.format(db_names));

## 3. Collections

### 3.1. Show all collections

In [ ]:
collection_names = db.list_collection_names()
print('* contain collections are: {}'.format(collection_names));

### 3.2. Create collection

In [ ]:
user_collection = db['user'];
print('* collection of "user" is: {}'.format(user_collection));

### 3.4. Insert data to collection

#### 3.4.1. Insert one data

In [ ]:
user = {'id': 1, 'name': 'Alvin', 'gender': 'M'}
r = user_collection.insert_one(user)
print('* insert user success, inserted id is: {}'.format(r.inserted_id))

#### 3.4.2. Insert many data

In [ ]:
users = [
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
]
r = user_collection.insert_many(users)
print('* insert user group success, inserted ids are: {}'.format(r.inserted_ids))

#### 3.4.3. Insert data with id

In [ ]:
user = {'_id': 'a', 'id': 4, 'name': 'Tom', 'gender': 'M'}
r = user_collection.insert_one(user)
print('* insert user success, inserted id is: {}'.format(r.inserted_id))

users = [
    {'_id': 'b', 'id': 5, 'name': 'Authur', 'gender': 'M'},
    {'_id': 'c', 'id': 6, 'name': 'Kate', 'gender': 'F'},
]
r = user_collection.insert_many(users)
print('* insert user group success, inserted ids are: {}'.format(r.inserted_ids))

### 3.5. Query collection

#### 3.5.1. Find all

In [ ]:
with user_collection.find() as rs:
    print('* find results are: \n{}'.format(format_result(rs)))

#### 3.5.2. find one

In [ ]:
rs = user_collection.find_one()
print('* find result is: \n{}'.format(format_result(rs)))

#### 3.5.3. find by properties

In [ ]:
with user_collection.find({}) as rs:
    print('* find result is: \n{}'.format(format_result(rs)))

with user_collection.find({'id': 1}) as rs:
    print('\n* find result is: \n{}'.format(format_result(rs)))

with user_collection.find({'id': 1, 'name': 'Alvin'}) as rs:
    print('\n* find result is: \n{}'.format(format_result(rs)))

#### 3.5.4. choose properties in result 

In [ ]:
with user_collection.find({'id': 1}, {'id': 1, 'name': 1}) as rs:
    print('\n* find result is: \n{}'.format(format_result(rs)))

#### 3.5.5. Condition operators

In [ ]:
with user_collection.find(
    {
        'id': {
            '$gt': 1,
            '$lte': 3
        }
    }
) as rs:
    print('* find result is: \n{}'.format(format_result(rs)))

with user_collection.find(
    {
        '$or': [
            {
                'id': {
                    '$gt': 1
                }
            },
            {
                'name': 'Alvin'
            }
        ]
    }
) as rs:
    print('\n* find result is: \n{}'.format(format_result(rs)))

with user_collection.find(
    {
        'id': {
            '$in': [2, 3]
        }
    }
) as rs:
    print('\n* find result is: \n{}'.format(format_result(rs)))

with user_collection.find(
    {
        'name': {
            '$regex': r'[A|E]\w+'
        }
    }
) as rs:
    print('\n* find result is: \n{}'.format(format_result(rs)))

### 3.6. Update collection

#### 3.6.1. Update whole collection

In [ ]:
with user_collection.find({'id': 1}) as rs:
    user = rs[0]
print('* find user is:\n{}'.format(format_result(user)))

user['name'] = 'Alvin'
user['address'] = 'Xi\'an'

user_collection.replace_one({'_id': user['_id']}, user)

with user_collection.find({'id': 1}) as rs:
    user = rs[0]
print('\n* find user is:\n{}'.format(format_result(user)))

#### 3.6.2. Update some property

In [ ]:
r = user_collection.update_one(
    {
        'id': 1
    },
    {
        '$set': {
            'name': 'Alvin.Q',
            'birthday': date_to_datetime(datetime.date(1981, 3, 17))
        },
        '$currentDate': {
            'lastModified': True
        }
    }
)
print('* {} data updaated'.format(r.modified_count))

rs = user_collection.find({'id': 1})
print('* after updated, data is:\n {}'.format(format_result(rs)))

#### 3.6.2. Update many collection

In [ ]:
r = user_collection.update_many(
    {
        'id': {
            '$gt': 1
        }
    },
    {
        '$set': {
            'type': 'STUDENT'
        },
        '$currentDate': {
            'lastModified': True
        }
    }
)
print('* {} data updaated'.format(r.modified_count))

rs = user_collection.find({'id': {'$gt': 1}})
print('* after updated, data is:\n {}'.format(format_result(rs)))

#### 3.6.3 Update with upsert 

In [ ]:
r = user_collection.update_one(
    {
        'id': 100
    },
    {
        '$set': {
            'name': 'Michael',
            'gender': 'M',
            'birthday': date_to_datetime(datetime.date(1981, 3, 17)),
            'type': 'STUDENT'
        },
        '$currentDate': {
            'lastModified': True
        }
    }, 
    upsert=True
)
if r.upserted_id:
    print('* update (with upsert) success, upserted id is: {}'.format(r.upserted_id))
else:
    print('* update (without upsert) success, modify count is: {}'.format(r.modified_count))
    
with user_collection.find({'id': 100}) as rs:
    print('* after update, find result is:\n{}'.format(format_result(rs)))

### 3.7. Delete

#### 3.7.1. Delete one

In [ ]:
r = user_collection.delete_one({'id': 100})
print('* delete count is: {}'.format(r.deleted_count))

## N. Close connection

In [ ]:
mongo.close()